In [2]:
import time
import getpass
username = getpass.getuser()
start_time = time.time()
#Where is the data?
if username == "bdutt":
    baseDir="/Users/bdutt/Documents/workspace-whitespace/face/face/data/lfw"
elif username =="cgarg":
    baseDir="/Users/cgarg/Documents/charu/intuit/code/face/data/lfw"
flow = "main"
debug = False

%run "Load Data.ipynb"

filenames = []
labels = []

def collectFileNamesAndLabels(filename):
    filenames.append(filename)
    result = filename.split("/") 
    labels.append(result[len(result)-2]) #Get the label name from filename
    

doForEachFile(collectFileNamesAndLabels,baseDir)
print("Total File name collection time = %s seconds" % (time.time() - start_time))
print ("Files collected : ",len(filenames))

Total File name collection time = 2.124769926071167 seconds
Files collected :  13233


In [3]:
start_time = time.time()
imagesList = []
doForEachFileNames(readImage, filenames, imagesList)
print ("Images data collected : ",len(imagesList))
print("Total data collection time = %s seconds" % (time.time() - start_time))

Images data collected :  13233


In [ ]:
start_time = time.time()

import pandas as pd
Y = pd.DataFrame(labels)
sizeOfYLabel = len(Y)
oneDImageList = []
for image in imagesList:
    oneDImageList.append(image.flatten())
X = pd.DataFrame(oneDImageList)
print("Total dataframe creation time = %s seconds" % (time.time() - start_time))

start_time = time.time()
%run "DNN.ipynb"

print("Total DNN time = %s seconds" % (time.time() - start_time))